In [6]:
import pandas as pd

purchase_behavior_path = r"C:\Users\kisho\OneDrive\Desktop\Kandi\purchase_behaviour.csv"
transaction_data_path = r"C:\Users\kisho\OneDrive\Desktop\Kandi\transaction_data.csv"

purchase_behavior = pd.read_csv(purchase_behavior_path)
transaction_data = pd.read_csv(transaction_data_path)

purchase_behavior_info = purchase_behavior.info()
purchase_behavior_head = purchase_behavior.head()

transaction_data_info = transaction_data.info()
transaction_data_head = transaction_data.head()

purchase_behavior_missing = purchase_behavior.isnull().sum()
transaction_data_missing = transaction_data.isnull().sum()

purchase_behavior_info, purchase_behavior_head, purchase_behavior_missing, transaction_data_info, transaction_data_head, transaction_data_missing


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72637 entries, 0 to 72636
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   LYLTY_CARD_NBR    72637 non-null  int64 
 1   LIFESTAGE         72637 non-null  object
 2   PREMIUM_CUSTOMER  72637 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264836 entries, 0 to 264835
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DATE            264836 non-null  int64  
 1   STORE_NBR       264836 non-null  int64  
 2   LYLTY_CARD_NBR  264836 non-null  int64  
 3   TXN_ID          264836 non-null  int64  
 4   PROD_NBR        264836 non-null  int64  
 5   PROD_NAME       264836 non-null  object 
 6   PROD_QTY        264836 non-null  int64  
 7   TOT_SALES       264836 non-null  float64
dtypes: float64(1), int64(6), object(1)
me

(None,
    LYLTY_CARD_NBR               LIFESTAGE PREMIUM_CUSTOMER
 0            1000   YOUNG SINGLES/COUPLES          Premium
 1            1002   YOUNG SINGLES/COUPLES       Mainstream
 2            1003          YOUNG FAMILIES           Budget
 3            1004   OLDER SINGLES/COUPLES       Mainstream
 4            1005  MIDAGE SINGLES/COUPLES       Mainstream,
 LYLTY_CARD_NBR      0
 LIFESTAGE           0
 PREMIUM_CUSTOMER    0
 dtype: int64,
 None,
     DATE  STORE_NBR  LYLTY_CARD_NBR  TXN_ID  PROD_NBR  \
 0  43390          1            1000       1         5   
 1  43599          1            1307     348        66   
 2  43605          1            1343     383        61   
 3  43329          2            2373     974        69   
 4  43330          2            2426    1038       108   
 
                                   PROD_NAME  PROD_QTY  TOT_SALES  
 0    Natural Chip        Compny SeaSalt175g         2        6.0  
 1                  CCs Nacho Cheese    175g         3 

In [7]:
transaction_data["DATE"] = pd.to_datetime(transaction_data["DATE"], origin="1899-12-30", unit="D")

merged_data = transaction_data.merge(purchase_behavior, on="LYLTY_CARD_NBR", how="left")

merged_data.head()


,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,LIFESTAGE,PREMIUM_CUSTOMER
0,2018-10-17,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0,YOUNG SINGLES/COUPLES,Premium
1,2019-05-14,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3,MIDAGE SINGLES/COUPLES,Budget
2,2019-05-20,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9,MIDAGE SINGLES/COUPLES,Budget
3,2018-08-17,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0,MIDAGE SINGLES/COUPLES,Budget
4,2018-08-18,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8,MIDAGE SINGLES/COUPLES,Budget


In [8]:
top_products = merged_data.groupby("PROD_NAME")["TOT_SALES"].sum().reset_index()

top_products = top_products.sort_values(by="TOT_SALES", ascending=False)

top_3_products = top_products.head(3)
top_3_products


,PROD_NAME,TOT_SALES
11,Dorito Corn Chp Supreme 380g,40352.0
86,Smiths Crnkle Chip Orgnl Big Bag 380g,36367.6
77,Smiths Crinkle Chips Salt & Vinegar 330g,34804.2


In [9]:
loyal_customers = merged_data.groupby(["LYLTY_CARD_NBR", "LIFESTAGE", "PREMIUM_CUSTOMER"])["TOT_SALES"].sum().reset_index()

loyal_customers = loyal_customers.sort_values(by="TOT_SALES", ascending=False)

top_loyal_segments = loyal_customers.groupby(["LIFESTAGE", "PREMIUM_CUSTOMER"])["TOT_SALES"].sum().reset_index()

top_loyal_segments = top_loyal_segments.sort_values(by="TOT_SALES", ascending=False)
top_loyal_segments.head(5)


,LIFESTAGE,PREMIUM_CUSTOMER,TOT_SALES
6,OLDER FAMILIES,Budget,168363.25
19,YOUNG SINGLES/COUPLES,Mainstream,157621.60
13,RETIREES,Mainstream,155677.05
15,YOUNG FAMILIES,Budget,139345.85
9,OLDER SINGLES/COUPLES,Budget,136769.80


# Hypothesis: Why Are These Customers Loyal?

## 1. Older Families (Budget Buyers)
- Likely buy in bulk for household needs.
- Prefer **budget-friendly** options to manage expenses.

## 2. Young Singles/Couples (Mainstream Buyers)
- Influenced by trends and convenience.
- Prefer snacks and ready-to-eat foods.

## 3. Retirees (Mainstream Buyers)
- Buy consistent, **trusted brands**.
- Prioritize quality over price.
